# <span style="color:#118DF0;">Historical Price Data Preparation</span>
To compute risk measures such as **Value-at-Risk (VaR)** and **Expected Shortfall (ES)**, we first need to obtain consistent historical data for the instruments in our portfolio. Since the portfolio contains multiple asset classes (equities, FX rates, commodities, bonds), we collected their historical market prices from multiple yearly files.  

The raw datasets contain daily information such as *Open*, *High*, *Low*, and *Close* prices for each instrument. For risk modeling, we specifically focus on the **Close price**, because it reflects the final traded value of the instrument at the end of each trading day and is widely used in return calculations.  

To prepare the dataset, the following steps were performed:  
- Import all yearly historical files.  
- Pivot the data so that each instrument has its own column.  
- Keep only the *Close* prices across all instruments.  
- Merge the yearly files into a single, consistent time series dataset.  
- Save the cleaned dataset in both CSV and Excel format.  

This consolidated dataset of close prices will serve as the foundation for computing **daily returns**, which are the inputs to the three VaR and ES estimation methods (historical simulation, parametric, and Monte Carlo).


In [1]:
import pandas as pd
import glob
import os
 
folder_path = r"C:\Users\ElMahdiAssab\Downloads\historical_prices\historical_prices"
 
# Only include yearly files, not the already merged one
csv_files = glob.glob(os.path.join(folder_path, "historical_prices_20*.csv"))
 
pivoted_dfs = []
 
for file in csv_files:
    df = pd.read_csv(file)
    # Pivot raw data (must have Date + Instrument + Close)
    pivot_df = df.pivot(index="Date", columns="Instrument", values="Close")
    pivot_df.columns.name = None   # remove the "Instrument" name
     
    pivot_df.index = pd.to_datetime(pivot_df.index)
    pivot_df.sort_index(inplace=True)
 
    pivoted_dfs.append(pivot_df)
 
# Merge all years
merged_df = pd.concat(pivoted_dfs)
merged_df = merged_df[~merged_df.index.duplicated(keep="first")]
merged_df.sort_index(inplace=True)
 
# # Save inside the same folder
# output_csv = os.path.join(folder_path, "historical_prices_all.csv")
# output_excel = os.path.join(folder_path, "historical_prices_all.xlsx")
 
# merged_df.to_csv(output_csv, index=True)
# merged_df.to_excel(output_excel, index=True)
 
print("Final merged dataset shape:", merged_df.shape)
merged_df

Final merged dataset shape: (1260, 22)


,AAPL,AUD/USD,Aus Gov 10Y,BHP,BMW,Bund 10Y,Can Gov 10Y,EUR/USD,GBP/USD,Gold,...,Nestle,RBC,Swiss Gov 10Y,Toyota,UK 10Y Gilt,US 10Y Treasury,USD/CAD,USD/CHF,USD/JPY,WTI Crude Oil
Date,,,,,,,,,,,,,,,,,,,,,
2020-10-19,70.323225,170.158713,95.017251,88.639465,183.177755,50.475668,87.289796,123.892541,194.635763,198.129937,...,108.235332,148.605138,186.954109,116.835225,93.921409,165.418681,56.257696,118.280793,170.147671,149.958816
2020-10-20,69.404005,170.634831,94.583179,88.814437,186.578282,50.648358,87.427433,123.959546,195.463770,200.717192,...,108.690467,148.179034,185.766414,116.515879,94.235642,165.776348,56.332972,117.917495,168.860751,152.648910
2020-10-21,69.334691,171.247707,94.609496,88.620642,186.044954,50.733245,87.164501,125.028863,196.736135,205.288732,...,108.910852,146.742350,185.549379,117.043194,94.578111,165.611650,55.857871,117.801254,167.942719,154.005691
2020-10-22,69.481287,169.352778,94.435659,88.168007,187.929328,50.871598,87.279964,125.760337,196.223812,208.369208,...,108.072712,142.995643,184.918542,117.676679,94.269199,165.073736,56.678525,119.560902,167.939504,153.156739
2020-10-23,67.940557,168.923113,94.442321,87.080900,185.737897,50.853006,87.312371,125.581356,198.979444,205.078620,...,109.289790,141.658133,185.923735,115.951192,94.353419,164.474595,56.415289,120.544325,168.644340,150.707661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-11,70.632357,124.978484,107.411225,142.343889,193.576027,74.390994,101.333095,176.288721,417.028127,154.285403,...,79.377454,125.346738,302.841636,136.911424,105.206651,203.004732,67.962944,157.850851,223.056545,67.407183
2025-08-12,70.804242,125.239417,107.794169,141.575567,191.285325,74.370275,101.228698,177.983841,415.726786,152.284748,...,81.129023,122.677836,303.245408,135.160051,105.502925,202.411381,68.080972,157.581738,224.351976,66.034407
2025-08-13,71.170940,126.003958,108.001812,142.442711,189.920745,74.047223,102.709684,176.555033,415.970667,152.004364,...,79.601350,122.207517,304.756634,135.820103,104.920802,201.296010,67.820741,158.781127,222.744090,64.953459


### **Data Refinement**

After preparing the consolidated dataset of historical close prices, the next step is to **align the dataset with the actual portfolio composition**.  
This involves two key adjustments:  

1. **Adding missing instruments**: Some assets in the portfolio (e.g., *MSFT* and *TSLA*) were not included in the original dataset and must be added by collecting their historical prices.  
2. **Removing unused instruments**: Assets that are not part of the portfolio are excluded from the dataset to ensure consistency and focus.  

The portfolio under study consists of 11 instruments with market values and weights, but for the risk analysis, we only keep the following relevant assets:  

- **MSFT**  
- **US 10Y Treasury**  
- **EUR/USD**  
- **WTI Crude Oil**  
- **Bund 10Y**  
- **Gold**  
- **USD/JPY**  
- **AAPL**  
- **TSLA**  

The detailed portfolio composition will be analyzed later, but at this stage, the dataset is cleaned and restricted to the required instruments only.


In [3]:
import yfinance as yf

# --- Portfolio instruments to keep ---
portfolio_instruments = [
    "MSFT", "US 10Y Treasury", "EUR/USD", "WTI Crude Oil",
    "Bund 10Y", "Gold", "USD/JPY", "AAPL", "TSLA"
]

# --- Download missing tickers (MSFT, TSLA) ---
new_prices = (
    yf.download(["MSFT", "TSLA"], start="2020-01-01", end="2025-08-15")["Close"]
    .rename_axis("Date")
)
new_prices.columns = new_prices.columns.get_level_values(-1)  # flatten MultiIndex if needed

# --- Merge & filter only portfolio instruments ---
final_df = (
    merged_df.join(new_prices, how="left")        # add MSFT & TSLA
    .filter(items=[c for c in portfolio_instruments if c in merged_df.columns or c in new_prices.columns])
    .sort_index()
)

print("Final dataset shape:", final_df.shape)
final_df


C:\Users\ElMahdiAssab\AppData\Local\Temp\ipykernel_13268\2175307292.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  yf.download(["MSFT", "TSLA"], start="2020-01-01", end="2025-08-15")["Close"]
[*********************100%***********************]  2 of 2 completed

Final dataset shape: (1260, 9)


,MSFT,US 10Y Treasury,EUR/USD,WTI Crude Oil,Bund 10Y,Gold,USD/JPY,AAPL,TSLA
Date,,,,,,,,,
2020-10-19,205.399551,165.418681,123.892541,149.958816,50.475668,198.129937,170.147671,70.323225,143.610001
2020-10-20,205.811874,165.776348,123.959546,152.648910,50.648358,200.717192,168.860751,69.404005,140.646667
2020-10-21,205.955704,165.611650,125.028863,154.005691,50.733245,205.288732,167.942719,69.334691,140.880005
2020-10-22,206.041977,165.073736,125.760337,153.156739,50.871598,208.369208,167.939504,69.481287,141.929993
2020-10-23,207.326797,164.474595,125.581356,150.707661,50.853006,205.078620,168.644340,67.940557,140.210007
...,...,...,...,...,...,...,...,...,...
2025-08-11,520.913696,203.004732,176.288721,67.407183,74.390994,154.285403,223.056545,70.632357,339.029999
2025-08-12,528.371399,202.411381,177.983841,66.034407,74.370275,152.284748,224.351976,70.804242,340.839996
2025-08-13,519.725647,201.296010,176.555033,64.953459,74.047223,152.004364,222.744090,71.170940,339.380005


## **Return Calculation**

Once we have the consolidated dataset of **Close prices**, the next step is to compute the **returns** of each instrument. Returns are essential for risk modeling because they measure the relative changes in value over time, rather than absolute price levels. This standardization allows us to compare different instruments across asset classes (equities, bonds, FX, commodities) on the same scale.  

There are two common approaches to compute returns:  
- **Simple Returns**:  
  $$
  R_t = \frac{P_t - P_{t-1}}{P_{t-1}}
  $$
  where \( P_t \) is the price at time \( t \).  

- **Logarithmic Returns**:  
  $$
  r_t = \ln \left( \frac{P_t}{P_{t-1}} \right)
  $$
  Log-returns are often preferred in risk management because they are time-additive and better approximate normality for small changes.  

In this project, we will use **logarithmic returns**, as they are more suitable for parametric VaR estimation and simulation-based methods.  

The result will be a new dataset where:  
- The **rows** correspond to dates.  
- The **columns** correspond to instruments.  
- The **values** are daily log-returns.  

This dataset of returns will serve as the direct input for the computation of **VaR and ES** using the historical, parametric, and Monte Carlo approaches.


In [4]:
import numpy as np

# ✅ final_df already contains Close prices for each instrument
# Make sure Date is the index
final_df.index = pd.to_datetime(final_df.index)

# ---- Simple Returns ----
simple_returns = final_df.pct_change().dropna()

# ---- Logarithmic Returns ----
log_returns = (final_df / final_df.shift(1)).dropna()

# ---- Save results ----
# simple_returns.to_csv(os.path.join(folder_path, "simple_returns.csv"))
# log_returns.to_csv(os.path.join(folder_path, "log_returns.csv"))

print("✅ Simple returns shape:", simple_returns.shape)
print("✅ Log returns shape:", log_returns.shape)

# Preview
log_returns

✅ Simple returns shape: (1259, 9)
✅ Log returns shape: (1162, 9)


C:\Users\ElMahdiAssab\AppData\Local\Temp\ipykernel_13268\1024833974.py:8: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  simple_returns = final_df.pct_change().dropna()


,MSFT,US 10Y Treasury,EUR/USD,WTI Crude Oil,Bund 10Y,Gold,USD/JPY,AAPL,TSLA
Date,,,,,,,,,
2020-10-20,0.002005,0.002160,0.000541,0.017780,0.003415,0.012974,-0.007592,-0.013158,-0.020850
2020-10-21,0.000699,-0.000994,0.008589,0.008849,0.001675,0.022521,-0.005451,-0.000999,0.001658
2020-10-22,0.000419,-0.003253,0.005833,-0.005528,0.002723,0.014894,-0.000019,0.002112,0.007425
2020-10-23,0.006216,-0.003636,-0.001424,-0.016120,-0.000366,-0.015918,0.004188,-0.022424,-0.012193
2020-10-26,-0.028854,-0.004355,0.000937,0.014890,-0.000232,0.011502,0.001017,0.013518,-0.000832
...,...,...,...,...,...,...,...,...,...
2025-08-08,0.002301,0.006811,-0.003874,-0.010491,-0.002200,-0.014463,0.009951,0.002499,0.022642
2025-08-11,-0.000517,-0.000276,0.004452,0.017529,0.003148,-0.041816,-0.003937,0.000574,0.028057
2025-08-12,0.014215,-0.002927,0.009570,-0.020576,-0.000279,-0.013052,0.005791,0.002431,0.005325


### **Portfolio Return Demonstration with Option Approximation**

### 1. General Definition  

The portfolio return is defined as:  

$$
R_p = \sum_{i=1}^{N} w_i R_i
$$

where:  
- $w_i$ is the weight of instrument $i$,  
- $R_i$ is its return,  
- $N$ is the total number of instruments.  



### 2. Application to the Portfolio  

Our portfolio contains:  

$$
\begin{aligned}
R_p \;=\;& w_{\text{AAPL Call}} \, R_{\text{AAPL Call}}
+ w_{\text{MSFT}} \, R_{\text{MSFT}}
+ w_{\text{USD/JPY}} \, R_{\text{USD/JPY}} \\
&+ w_{\text{WTI}} \, R_{\text{WTI}}
+ w_{\text{AAPL}} \, R_{\text{AAPL}}
+ w_{\text{EUR/USD}} \, R_{\text{EUR/USD}} \\
&+ w_{\text{EUR/USD Call}} \, R_{\text{EUR/USD Call}}
+ w_{\text{Bund 10Y}} \, R_{\text{Bund 10Y}} \\
&+ w_{\text{Gold}} \, R_{\text{Gold}}
+ w_{\text{US 10Y Treasury}} \, R_{\text{US 10Y Treasury}} \\
&+ w_{\text{TSLA}} \, R_{\text{TSLA}}
\end{aligned}
$$

### 3. Option Approximation Using **Delta**  

The return of an option can be approximated by:  

$$
R_{\text{Option}} \approx \Delta \cdot R_{\text{Underlying}}
$$

So:  
- $R_{\text{AAPL Call}} \approx \Delta_{\text{AAPL Call}} \cdot R_{\text{AAPL}}$,  
- $R_{\text{EUR/USD Call}} \approx \Delta_{\text{EUR/USD Call}} \cdot R_{\text{EUR/USD}}$.  

### 4. Substituting Into the Portfolio Return  

$$
\begin{aligned}
R_p \;=\;& w_{\text{AAPL Call}} \, (\Delta_{\text{AAPL Call}} \cdot R_{\text{AAPL}}) 
+ w_{\text{MSFT}} \, R_{\text{MSFT}}
+ w_{\text{USD/JPY}} \, R_{\text{USD/JPY}} \\
&+ w_{\text{WTI}} \, R_{\text{WTI}}
+ w_{\text{AAPL}} \, R_{\text{AAPL}}
+ w_{\text{EUR/USD}} \, R_{\text{EUR/USD}} \\
&+ w_{\text{EUR/USD Call}} \, (\Delta_{\text{EUR/USD Call}} \cdot R_{\text{EUR/USD}}) \\
&+ w_{\text{Bund 10Y}} \, R_{\text{Bund 10Y}}
+ w_{\text{Gold}} \, R_{\text{Gold}} \\
&+ w_{\text{US 10Y Treasury}} \, R_{\text{US 10Y Treasury}}
+ w_{\text{TSLA}} \, R_{\text{TSLA}}
\end{aligned}
$$

### 5. Factorization by Underlying  

$$
\begin{aligned}
R_p \;=\;& 
\underbrace{\big( w_{\text{AAPL}} + w_{\text{AAPL Call}} \, \Delta_{\text{AAPL Call}} \big)}_{\text{Net exposure to AAPL}} \, R_{\text{AAPL}} \\
&+ w_{\text{MSFT}} \, R_{\text{MSFT}} \\
&+ \underbrace{\big( w_{\text{EUR/USD}} + w_{\text{EUR/USD Call}} \, \Delta_{\text{EUR/USD Call}} \big)}_{\text{Net exposure to EUR/USD}} \, R_{\text{EUR/USD}} \\
&+ w_{\text{USD/JPY}} \, R_{\text{USD/JPY}} \\
&+ w_{\text{WTI}} \, R_{\text{WTI}} \\
&+ w_{\text{Bund 10Y}} \, R_{\text{Bund 10Y}} \\
&+ w_{\text{Gold}} \, R_{\text{Gold}} \\
&+ w_{\text{US 10Y Treasury}} \, R_{\text{US 10Y Treasury}} \\
&+ w_{\text{TSLA}} \, R_{\text{TSLA}}
\end{aligned}
$$

#### Matrix Notation

Let:  

$$
\mathbf{w} = 
\begin{bmatrix}
w_{\text{AAPL}} + w_{\text{AAPL Call}} \Delta_{\text{AAPL Call}} \\
w_{\text{MSFT}} \\
w_{\text{EUR/USD}} + w_{\text{EUR/USD Call}} \Delta_{\text{EUR/USD Call}} \\
w_{\text{USD/JPY}} \\
w_{\text{WTI}} \\
w_{\text{Bund 10Y}} \\
w_{\text{Gold}} \\
w_{\text{US 10Y Treasury}} \\
w_{\text{TSLA}}
\end{bmatrix}, 
\quad
\mathbf{R} =
\begin{bmatrix}
R_{\text{AAPL}} \\
R_{\text{MSFT}} \\
R_{\text{EUR/USD}} \\
R_{\text{USD/JPY}} \\
R_{\text{WTI}} \\
R_{\text{Bund 10Y}} \\
R_{\text{Gold}} \\
R_{\text{US 10Y Treasury}} \\
R_{\text{TSLA}}
\end{bmatrix}
$$

Then the portfolio return can be compactly written as:  

$$
R_p = \mathbf{w}^\top \mathbf{R}
$$




# <span style="color:#118DF0;">Exploratory Data Analysis of the Portfolio</span>

Before applying risk measures such as **VaR** and **Expected Shortfall**, it is essential to thoroughly understand the structure and composition of the portfolio. The dataset provided contains detailed information about each position, including both standard assets (equities, bonds, FX, commodities) and derivative instruments (options).

The available columns are:
- **Position_Id**: Unique identifier for each position in the portfolio.  
- **Instrument_Type**: Type of financial instrument (Equity, Bond, FX, Option, etc.).  
- **Instrument_Name**: Specific instrument name (e.g., AAPL, EUR/USD, WTI Crude Oil).  
- **Asset_Class**: Asset category (Equities, Fixed Income, FX, Commodities, etc.).  
- **Currency**: Denomination currency of the position.  
- **Notional**: Face value of the contract (for derivatives) or exposure amount.  
- **Market_Value**: Current value of the position in portfolio currency.  
- **Greeks (Delta, Gamma, Vega, Theta, Rho)**: Sensitivity measures relevant for derivative instruments.

Since the portfolio contains **11 distinct instruments**, each with potentially multiple positions, the objectives of this EDA are:
1. **Check dataset structure** and identify missing or inconsistent data.  
2. **Understand portfolio composition** by asset class, instrument type, and currency.  
3. **Aggregate positions** at the instrument level to get total market exposure.  
4. **Visualize exposures** across instruments and asset classes.  
5. **Analyze sensitivities (Greeks)** to assess the risk profile of derivative positions.  

This analysis will provide the necessary foundation before simulating returns and computing **risk measures**.


In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the portfolio file
portfolio_path = r"C:\Users\ElMahdiAssab\Downloads\large_trading_portfolio_with_sensitivities.csv"
portfolio_df = pd.read_csv(portfolio_path)

print("✅ Portfolio dataset loaded successfully")
print("Shape:", portfolio_df.shape)
print("\nPreview of the dataset:")
portfolio_df

✅ Portfolio dataset loaded successfully
Shape: (10000, 13)

Preview of the dataset:


,Position_Id,Instrument_Type,Instrument_Name,Asset_Class,Currency,Notional,Market_Value,Delta,Volatility,Gamma,Vega,Theta,Rho
0,MSFT-001,Equity,MSFT,Equity,USD,35604840,-6.166015e+06,-1.423825,0.294724,-0.000289,-0.265307,-0.000680,0.006969
1,US 10Y Treasury-001,Bond,US 10Y Treasury,Rates,USD,3694128,8.225396e+06,0.758237,0.051669,0.000016,0.006851,-0.001144,0.099530
2,US 10Y Treasury-002,Bond,US 10Y Treasury,Rates,USD,20366766,-9.614017e+06,-0.522397,0.441047,0.000009,-0.019552,-0.003771,0.016652
3,AAPL Call Option-001,Equity Option,AAPL Call Option,Equity,USD,23766269,3.068638e+06,-0.207339,0.360725,0.018143,0.577026,-0.210032,-0.044319
4,EUR/USD-001,FX Forward,EUR/USD,FX,USD,3995090,-2.311821e+06,-0.090412,0.565649,0.000742,-0.048947,-0.001323,0.046593
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,US 10Y Treasury-939,Bond,US 10Y Treasury,Rates,USD,23660264,9.959134e+06,-0.133925,0.212878,-0.000067,-0.016842,-0.002342,-0.256171
9996,MSFT-909,Equity,MSFT,Equity,USD,9533341,6.393792e+05,0.722844,0.248343,0.001818,0.649715,0.009486,-0.009580
9997,AAPL-950,Equity,AAPL,Equity,USD,27129442,-2.495172e+06,0.079350,0.184371,0.000709,-0.106267,-0.031820,0.000738
9998,WTI Crude Oil-889,Commodity Future,WTI Crude Oil,Commodity,USD,41885230,-2.282784e+06,-0.473929,0.568420,0.001398,0.671080,0.000054,0.001874


In [7]:
# Basic dataset inspection

print("\n--- Dataset Info ---")
print(portfolio_df.info())

print("\n--- Descriptive Statistics (numerical + categorical) ---")
display(portfolio_df.describe(include='all'))

print("\n--- Missing Values ---")
print(portfolio_df.isnull().sum())


--- Dataset Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Position_Id      10000 non-null  object 
 1   Instrument_Type  10000 non-null  object 
 2   Instrument_Name  10000 non-null  object 
 3   Asset_Class      10000 non-null  object 
 4   Currency         10000 non-null  object 
 5   Notional         10000 non-null  int64  
 6   Market_Value     10000 non-null  float64
 7   Delta            10000 non-null  float64
 8   Volatility       10000 non-null  float64
 9   Gamma            10000 non-null  float64
 10  Vega             10000 non-null  float64
 11  Theta            10000 non-null  float64
 12  Rho              10000 non-null  float64
dtypes: float64(7), int64(1), object(5)
memory usage: 1015.8+ KB
None

--- Descriptive Statistics (numerical + categorical) ---


,Position_Id,Instrument_Type,Instrument_Name,Asset_Class,Currency,Notional,Market_Value,Delta,Volatility,Gamma,Vega,Theta,Rho
count,10000,10000,10000,10000,10000,1.000000e+04,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
unique,10000,6,11,4,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,MSFT-907,Equity,AAPL Call Option,Equity,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,2755,954,3709,9080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,2.499994e+07,1.821735e+05,0.001197,0.323834,0.000083,-0.003911,-0.023468,-0.000280
std,NaN,NaN,NaN,NaN,NaN,1.436630e+07,5.767764e+06,0.950283,0.159036,0.007893,0.745135,0.073540,0.175745
min,NaN,NaN,NaN,NaN,NaN,1.032120e+05,-9.999038e+06,-3.746006,0.050009,-0.089887,-6.170094,-0.604599,-1.572100
25%,NaN,NaN,NaN,NaN,NaN,1.234818e+07,-4.745414e+06,-0.591440,0.186710,-0.000578,-0.154306,-0.018984,-0.017946
50%,NaN,NaN,NaN,NaN,NaN,2.501425e+07,2.407931e+05,0.005982,0.322491,0.000019,0.000168,-0.005551,0.000301
75%,NaN,NaN,NaN,NaN,NaN,3.737866e+07,5.157383e+06,0.597736,0.461913,0.000648,0.148798,-0.000009,0.017559



--- Missing Values ---
Position_Id        0
Instrument_Type    0
Instrument_Name    0
Asset_Class        0
Currency           0
Notional           0
Market_Value       0
Delta              0
Volatility         0
Gamma              0
Vega               0
Theta              0
Rho                0
dtype: int64


## **Portfolio composition analysis**

Before computing risk measures, we analyze the portfolio composition to understand exposure, concentration and currency risks. The following subsections present visual summaries and explain how weights are computed.


### Exposure by Asset Class

In [8]:
import plotly.express as px

def plot_pie(data, title):
    colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3']  # couleurs fixes
    fig = px.pie(
        names=data.index,
        values=data.values,
        title=title,
        color=data.index,  # permet d'appliquer les couleurs aux labels
        color_discrete_sequence=colors
    )
    fig.update_traces(textposition='inside', textinfo='percent+label', pull=[0.02]*len(data))
    fig.update_layout(
        title={'x':0.5, 'xanchor':'center'},
        showlegend=False,
        margin=dict(t=40, b=10, l=10, r=10)
    )
    fig.show()
# Agrégation par Asset Class
asset_class_dist = portfolio_df.groupby("Asset_Class")["Market_Value"].sum().sort_values(ascending=False)

# asset_class_dist
# Visualisation
plot_pie(asset_class_dist, "Portfolio Exposure by Asset Class")

This chart aggregates the **Market_Value** of all positions by `Asset_Class` and displays each class’ share of the portfolio.

**How weights are computed:**  
Let $MV_i$ be the market value of asset class $i$ expressed in the portfolio base currency (see currency note below).

The weight of class $i$ is:
$$
w_i \;=\; \frac{MV_i}{\sum_j MV_j}
$$

**What to look for:** concentration (one class > 50%), diversification (balanced shares), and implication on risk drivers (e.g., equities vs fixed income).

- The portfolio is **heavily weighted in equities**, which account for nearly half of the total market value. This indicates a significant exposure to equity market movements.  
- **FX and commodities** together form a substantial portion (~50%), suggesting diversification across asset types, but still leaving equities as the dominant risk driver.  
- **Rates exposure is minimal** (2%), implying that interest rate movements have limited direct impact on portfolio value.  
- Overall, the portfolio shows **moderate diversification**, but the concentration in equities highlights potential vulnerability to market shocks in the equity space.  
- Investors should consider whether this level of concentration aligns with their **risk tolerance and investment objectives**.

### Exposure by Instrument Type

In [9]:
# Agrégation par Instrument Type
instrument_type_dist = portfolio_df.groupby("Instrument_Type")["Market_Value"].sum().sort_values(ascending=False)

# Visualisation
plot_pie(instrument_type_dist, "Portfolio Exposure by Instrument Type")

This chart groups exposures by `Instrument_Type` (Equity, Option, Bond, FX, Commodity, …).

**Why this matters:** instrument type determines risk behavior: derivatives (options, swaps) introduce non-linear risk (Gamma, Vega), while equities are linear exposures.

**Weight calculation:** same formula as above but aggregating by `Instrument_Type`:
$$
w_{\text{type}} = \frac{\sum_{p \in \text{type}} MV_p}{\sum_{q} MV_q}
$$


- The portfolio shows a **balanced exposure across major instrument types**: FX, equities, and equity options each represent ~23–24% of the portfolio.  
- **Derivatives (Equity Option, FX Option)** account for nearly 28% of total exposure, indicating the presence of **non-linear risk** (Gamma, Vega), which could amplify gains or losses in volatile markets.  
- **Commodity Futures** contribute a significant portion (20.5%), adding diversification outside traditional financial instruments.  
- **Bonds** remain a minor exposure (2%), implying limited sensitivity to interest rate changes.  
- Overall, the portfolio demonstrates **moderate diversification by instrument type**, but the sizeable derivative exposure warrants attention to potential non-linear risk effects.

### Exposure by Currency

In [10]:
# Agrégation par Currency
currency_dist = portfolio_df.groupby("Currency")["Market_Value"].sum().sort_values(ascending=False)

# Visualisation
plot_pie(currency_dist, "Portfolio Exposure by Currency")

This pie shows the share of portfolio value per `Currency`. All values **must** be converted to the portfolio base currency before aggregation.

**Currency conversion (if needed):** for a position quoted in local currency:
$$
MV^{\text{base}} = MV^{\text{local}} \times FX_{\text{local}\to\text{base}}
$$

After conversion, compute weights as:
$$
w_{\text{currency}} = \frac{\sum_{p \in \text{currency}} MV^{\text{base}}_p}{\sum_q MV^{\text{base}}_q}
$$

**Interpretation:** a concentrated currency exposure implies FX risk; hedging decisions may follow.


### Exposure by Instrument 

In [11]:
# === 1. FX conversion ===
EUR_USD = 1.1705  # EUR/USD as of 16 Aug 2025

portfolio_df["Market_Value_USD"] = portfolio_df.apply(
    lambda row: row["Market_Value"] * (EUR_USD if row["Currency"] == "EUR" else 1.0),
    axis=1
)

# === 2. Aggregate by Instrument ===
# On définit les colonnes à sommer et celles à garder (ex: Asset_Class, Instrument_Type)
numeric_cols = ["Notional", "Market_Value", "Market_Value_USD", "Delta", "Volatility", "Gamma", "Vega", "Theta", "Rho"]
non_numeric_cols = ["Instrument_Type", "Asset_Class", "Currency"]  # on prend la première valeur

instrument_agg = portfolio_df.groupby("Instrument_Name").agg(
    {**{col: "sum" for col in numeric_cols}, **{col: "first" for col in non_numeric_cols}}
).sort_values("Market_Value_USD", ascending=False)

# === 3. Compute weights ===
total_mv_usd = instrument_agg["Market_Value_USD"].sum()
weights = instrument_agg["Market_Value_USD"] / total_mv_usd
instrument_agg["Weight (%)"] = (weights * 100).round(2)

# === 4. Compute HHI (optionnel) ===
hhi = (weights ** 2).sum()
if hhi < 0.10:
    interpretation = "The portfolio is well diversified."
elif hhi < 0.18:
    interpretation = "The portfolio shows moderate concentration."
else:
    interpretation = "The portfolio is highly concentrated."

# === 5. Display table ===
print("=== Portfolio Composition by Instrument (Aggregated) ===")
print(instrument_agg.reset_index())
print("\nTotal Market Value (USD):", round(total_mv_usd, 2))
print("Herfindahl–Hirschman Index (HHI):", round(hhi, 4))
print("Interpretation:", interpretation)

# === 6. Visualization ===
fig = px.bar(
    instrument_agg.reset_index(),
    x="Instrument_Name",
    y="Market_Value_USD",
    text="Weight (%)",
    title="Portfolio Composition by Instrument (USD, 16 August 2025)",
    labels={"Market_Value_USD": "Market Value (USD)", "Instrument_Name": "Instrument"}
)

fig.update_traces(
    texttemplate="%{text:.2f}%",
    textposition="outside",
    marker_color="steelblue"
)
fig.update_layout(
    xaxis_tickangle=45,
    margin=dict(t=60, b=100),
    showlegend=False
)
fig.show()


=== Portfolio Composition by Instrument (Aggregated) ===
        Instrument_Name     Notional  Market_Value  Market_Value_USD  \
0      AAPL Call Option  23357713722  4.255906e+08      4.255906e+08   
1                  MSFT  22967416466  4.100473e+08      4.100473e+08   
2               USD/JPY  22045141568  3.440054e+08      3.440054e+08   
3         WTI Crude Oil  22248804716  2.953048e+08      2.953048e+08   
4                  AAPL  24068347785  1.152913e+08      1.152913e+08   
5               EUR/USD  21681510035  1.046561e+08      1.046561e+08   
6   EUR/USD Call Option  21475661348  9.792177e+07      9.792177e+07   
7              Bund 10Y  23213978252  7.924353e+07      9.275456e+07   
8                  Gold  22489000202  8.126269e+07      8.126269e+07   
9       US 10Y Treasury  23899158993 -4.189366e+07     -4.189366e+07   
10                 TSLA  22552636810 -8.969499e+07     -8.969499e+07   

        Delta  Volatility     Gamma       Vega       Theta       Rho  \
0    9

Display the composition of the portfolio by instrument, with all market values converted into **USD** as of **16 August 2025**.

**Why:** identifies single-instrument concentration and key risk contributors.

**Weight and concentration metrics:**  
- Instrument weight:  
  $$
  w_i = \frac{MV_i^{USD}}{\sum_j MV_j^{USD}}
  $$  
- Herfindahl–Hirschman Index (HHI) for concentration:  
  $$
  \text{HHI} = \sum_i w_i^2
  $$  
  (Higher HHI → higher concentration. The HHI is particularly useful to quantify **portfolio diversification**, by showing whether the portfolio is dominated by a few instruments or more evenly distributed.)

**Note on FX conversion:**  
Since the date of valuation is 16 August 2025, we use the exchange rate **1 EUR = 1,1705 USD** to convert all EUR market values into USD.  
